This problem was asked by Facebook.

You are given an array of non-negative integers that represents a two-dimensional elevation map where each element is unit-width wall and the integer is the height. Suppose it will rain and all spots between two walls get filled up.

Compute how many units of water remain trapped on the map in O(N) time and O(1) space.

For example, given the input [2, 1, 2], we can hold 1 unit of water in the middle.

Given the input [3, 0, 1, 3, 0, 5], we can hold 3 units in the first index, 2 in the second, and 3 in the fourth index (we cannot hold 5 since it would run off to the left), so we can trap 8 units of water.

This is an extrema question! Yes, calculus!

Walls of each "bucket" of water are local maxes. Can we identify local maxes in O(N) time and O(1) space? Sure, easily: we look for sign changes. 

Consider three wall heights: a,b,c, in order. If h_b>h_a, and h_b>h_c, then h_b is a local max, and so a wall for our bucket. Let's write a function to identify local maxes.

In [6]:
#return endpoints and local max indices
def max_indices(arr):
    if len(arr)<=1: return [0]
    maxes=[0]
    for i in range(1,len(arr)-1):
        if arr[i]>=arr[i-1] and arr[i]>arr[i+1]: maxes.append(i)
    maxes.append(len(arr)-1)
    return maxes

    

In [7]:
ex = [2,1,2]
max_indices(ex)

[0, 2]

In [8]:
ex = [3,0,1,3,0,5]
max_indices(ex)

[0, 3, 5]

In [9]:
ex = [2,0,4,4,0,2]
max_indices(ex)

[0, 3, 5]

In [5]:
max_indices([7])

[0]

Ok, so now that we can grab the walls between which water is held, we can compute how much water can be held by each bucket (gap between maxes). This amount of water is min(local_max1,local_max2)-wallheight_i for every i between the two max indices.

This won't work, though, for a set of walls like [3,0,1,0,3]. The idea above will fill 1 and 1, when the answer should be fill 3,2,3. Hm.

We can use the above method and then literally fill the gaps, iteratively, and run it again. Like how water really fills the gaps. In the example above, 

[3,0,1,0,3] --> [3,1,1,1,3] --> [3,3,3,3,3]

Or rather, I want it to work this way, but with how I've written max_indices, each 1 will be identified as a max and so none of those gaps will be filled. Hm.

In [10]:
ex = [3,1,1,1,3]
max_indices(ex)

[0, 4]

I changed max indices so that it will only recognize the last max in a set of repeated maxes: ie, in [2,0,1,1,0,4] it will only recognize the second one. This change also means, in [2,1,1,3] none of the ones will be recognized as maxes.

So, idea: fill every index between two local maxes (inclusive) with an amount of water equal to the min(left_max_ht,right_max_ht). This should correspond to the first water buckets that get filled. If this works, I'll worry about the second filling.

Actually, let's worry about that now. If there are n walls, then I'll have to do a second filling, maybe a third, maybe a fourth, up to possible n-1 fillings. That means my runtime will be O(n^2). Not good enough.

Let's think about the problem in a different way (and thanks to GeeksforGeeks for the intuition). Every index in the arr will eventually be part of a bucket of water whose left wall will be the tallest wall to its left and whose right wall will be the tallest wall to its right. It should be pretty easy to compute the tallest wall to the left and right for any index i. Let's do that in two loops, filling left_walls and right_walls.

Once we have the wall heights, then by the same logic as before, each index gets filled to the min of (left_wall, right_wall). So let's "fill" them one by one and keep track of the amount of water that gets used: (min(l,r)-height).

In [13]:
def fill(walls):
    n = len(walls)
    left = [0 for i in range(n)]
    right = [0 for i in range(n)]
    
    for i in range(n):
        left[i] = max(left[i-1],walls[i])
    
    
    for i in range(-1,-1*n-1,-1):
        right[i] = max(right[i+1],walls[i])
    
    water = 0
    
    for i in range(n):
        water += min(left[i],right[i])-walls[i]
    
    return water


In [14]:
ex = [2,1,2]
fill(ex)

1

In [15]:
ex = [3,0,1,3,0,5]
fill(ex)

8

In [16]:
ex = [7]
fill(ex)

0